# Quantopian でできること

"Tokyo Quantopian Users Group" と縁の深い"Fin-py"コミュニティーのもくもく会に参加される方の中には「自作のシステムトレーディングツールを作る」ということをテーマにされる方が一定数います。

本日参加されている方の中にもそういうシステムを作っている、あるいはQuantopian以外のシステムを使っている、という人もいらっしゃるかと思います。

もし仮に、自分でそのようなシステムを作るとするならば、どのようなトレーディングツールを設計するでしょうか？ アルゴリズムトレーディングシステムの概要をイメージした上で、このNotebookのテーマである「Quantopianでできること」を学習したいと思います。

## 基本的なアルゴリズムの枠組み

おそらく、自作をする場合、以下のような構成が一般的なものになると思います。

1. 一定期間ごとにアルゴリズム（売買プログラム）を起動する
2. 稼働したタイミングにおけるマーケット（投資対象群）のデータを取得する（現在値段、取引数量、財務データなど）
3. 取得したデータを元に、シグナルを計算する。その値に基づいて売買を実行する
4. 1.に戻る（繰り返し）

どうでしょうか？だいたいこんな感じのものをイメージするのではないでしょうか？
それぞれの機能について、掘り下げていきましょう。

## 1．一定期間毎にアルゴリズムを稼働させる

トレードの起点となります。この処理を呼び出すことで、投資判断が始まります。
まず第一に、この処理の起点を**どれくらいの頻度で呼び出すか？**というのが、アルゴリズムを作成する上での最初の意思決定となるでしょう。

この意思決定を行うにあたり、2つの判断材料があると思います

### システム性能

「5分間隔ごとにアルゴリズムを稼働させたい」と思っても、株価データが日足しかない、、、という場合は1日に一回しかアルゴリズムを回すことができません（正確に言えば、1日に何回も回すことはできますが、データの更新が発生しないため結果が変化しません）。

したがって、アルゴリズムの稼働間隔は、システム性能によって制約を受けます。
Quantopianの場合、１分間隔で市場データを提供していますので、一番粒度の細かいアルゴリズム稼働間隔は１分間に一度、となります。

個人でやる分には十分な株価データが揃っていると言えますが、「１カイ２ヤリ」と呼ばれるような細かい値動きで収益をあげたり、**HFT**(High Frequency Trading : 高頻度売買)と呼ばれるミリ秒単位で高速に売買を行うようなシステムの構築はQuantopiantといえどもできない、ということです。

### 投資戦略（タイムホライズン）

投資に対するスパンをどのように捉えているか？もアルゴリズムの稼働間隔に影響を与えると考えられます。これは「ファンダメンタル分析」「テクニカル分析」のどちらに基づいて株式売買を行うか、という点にも通じます。

ファンダメンタル分析の場合、投資対象の財務データや企業情報などに基づいて投資判断を行いますが、こうしたデータは1日に何度も変化しません。財務データであれば、3ヶ月に１度（四半期報告）程度です。企業「価値」に基づいて投資を行うのであれば、1日に何度もアルゴリズムを稼働させる必要はないでしょう。

テクニカル分析の場合、企業「価値」よりも、その時どきの需給関係に基づいた「価格」に基づいて投資判断を行うので、取引時間中に発生する、細かい需給の変化をアルゴリズムで補足して売買判断をすることも考えられます。

----

以上のような観点から、アルゴリズムの稼働間隔を決めるのが第一歩と言えるでしょう。さて今回のテーマである、ファンダメンタルファクターモデルでのアルゴリズム稼働間隔はどの程度が適当でしょうか？

## 2. マーケット（投資対象群）のデータを取得して、シグナルを計算する

アルゴリズムが稼働した時点において取得可能な情報を集めます。情報を集めるに当たっても2つの判断材料があると考えられます。

* **何に投資するのか？**

投資対象となる銘柄郡の情報を取得します。金融の世界において、投資対象となる銘柄群のことを「ユニバース」と呼びます。どのような投資を行いたいかによってユニバースを決定します。広くは、

* 株式
* 債券
* コモディティ（商品）

という枠組みも「ユニバース」です。しかしながら、「ユニバース」を広くとると、情報収集にかかるコストが増大するため運用効率が下がるうえ、実際には取引が難しい（ほとんど取引が成立していない銘柄）などを含んでしまうデメリットがあります。そこで

* **米国株のうち、流動性が高い銘柄**

というように、投資対象を絞って、実際に投資が行えるような現実的なユニバースを選択することが一般的です。

* **どういう情報に基づいて投資判断を行うのか？**

前述の「投資スタイル」でも説明したように、「ファンダメンタル分析」や「テクニカル分析」のどちらを用いるかによって異なります。

* ファンダメンタル分析：財務データ、企業情報など
* テクニカル分析：相場情報（過去の値動き、出来高）など

これらを、アルゴリズムが稼働した時点を基準とし、その時点で利用可能なデータから、分析に必要なデータを集めます。
分析に必要なデータがそろえば、そのデータを元に、売買判断を行う基準となる「シグナル」を計算するつことができます。


## 3. シグナルに基づいて売買を実行する

最後にシグナルの値に基づいて、

* どの株を
* 何株
* いくらで
* 買う|売る

のか、を決定します。「いくらで」の部分については、シグナル計算時点に取得できているデータに基づいて割安/割高判断がついているので、指値指定をせず、成行注文にしてしまう場合が殆どだと思います。

## Quantopian の機能との対応
ここまで、「一般的にアルゴリズムトレーディングに求められるシステム」についての機能を考察してきました。もちろんQuantopianでもこれらの機能が実現されています。ざっくりと対応表を作ると、以下のような感じになるかと思います。


|基本的な枠組み|Quantopianでの対応機能|
|:----:|:----:|
|1.一定期間毎にアルゴリズムを稼働させる|Quantopian.Algorithm.schedule_function()|
|2.マーケット（投資対象群）のデータを取得して、シグナルを計算する|**Quantopian.Pipeline()**|
|3.シグナルにもどついて売買を実行する|Quantopian.Algorithm.Order()|

## シグナルについて

本日の勉強会においては、ファクターモデルに基づいてシグナルを決定が決定されるため、本日に限って言えば、**シグナル＝ファクター**ということができます。

# Part2.の目標
Part2.では、データ収集とシグナル計算を担っている**パイプライン**の使い方とともに、ファクターの構築方法について学習します。
作成したファクターの評価方法については、Part3.で触れていきます。